In [29]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import warnings
warnings.filterwarnings('ignore')

from folium import plugins
import json
import requests
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

import seaborn as sns
import matplotlib.pyplot as plt

import dash
import dash_core_components as dcc
import dash_html_components as html

from jupyter_plotly_dash import JupyterDash

import time

import plotly
import numpy as np

In [2]:
# load data
data=pd.read_excel('I:/Brandschadenstatistik_Recycling.xlsx')
data=data.drop(index=[0,1])
data=data.reset_index(drop=True)
data.isna().sum()

Nr.                                       0
Tag                                       2
Datum                                     0
Uhrzeit                                  39
Firma                                    47
Land                                      0
Ort                                       1
Brandentstehungsort                      18
Brandursache                             10
Brandherd                                16
Art des in Brand geratenen Materials     56
Branderkennung durch                    191
Unnamed: 12                             272
Unnamed: 13                             208
Unnamed: 14                             242
Brandbekämpfung                         254
Unnamed: 16                             268
Unnamed: 17                              20
Folgeschaden                            268
Brand-\nvorschäden                      226
Schadenhöhe                             213
Link                                      5
Unnamed: 22                     

In [3]:
# pickup the useful information 
df=data[['Datum','Uhrzeit','Firma','Ort','Brandentstehungsort','Brandherd','Art des in Brand geratenen Materials']]
df.head()

,Datum,Uhrzeit,Firma,Ort,Brandentstehungsort,Brandherd,Art des in Brand geratenen Materials
0,2019-06-16,22:00:00,Berliner Stadtreinigung,Berlin Neukölln,Lagerhalle,Lagergut,"6000 Tonnen Haus- und Sperrmüll, Müllbunker"
1,2020-02-15,09:00:00,Alfred Hösli AG,Weesen,Halle,Arbeitsmaschinen,Abbruchfahrzeuge
2,2018-08-18,NaN,CRH Recycling,Crailsheim,Fahrzeug,Müllpresswagen,"Abfall aus Müllwagen, Schaden am Asphalt"
3,2019-09-04,NaN,Mittelstädt,Uslar,Freilager,Lagergut,Altholz
4,2019-05-13,12:50:00,Heidemann Recycling GmbH,Düdenbüttel,Freilager,Lagergut,Altholz und Bauschutt


In [4]:
df_geo=df.copy()

# get longtitude and latitude
locator=Nominatim()
geocode=RateLimiter(locator.geocode)
df_geo['location']=df_geo['Ort'].apply(geocode)
df_geo['point']=df_geo['location'].apply(lambda loc:tuple(loc.point) if loc else None)
df_geo[['latitude','logitude','altitude']]=pd.DataFrame(df_geo['point'].tolist(),index=df.index)
df_geo=df_geo.drop(['location','point'],axis=1)
df_geo=df_geo.dropna(subset=['latitude','logitude'])                                

In [227]:
# overveiw of location
def my_map_overview(df_geo):
    my_map=folium.Map(
    location=[48,14],
    tiles='cartodbpositron',
    zoom_start=4,
    width=600,
    height=400
    )
    
    latitudes=list(df_geo.latitude)
    longitudes=list(df_geo.logitude)
    labels=list(df_geo.Firma)
    
    for lat,lng,label in zip(latitudes,longitudes,labels):
        folium.CircleMarker([lat,lng],popup=label,radius=1,color='red',fill_color='yellow').add_to(my_map)
    
    incidents=folium.map.FeatureGroup()
    my_map.add_child(incidents)
    
    # add EU border

    EU_geo = 'I:/EU.geo.json'
    folium.GeoJson(
    EU_geo,
    style_function=lambda feature: {
        'fillColor': '#ffff00',
        'color': 'blue',
        'weight': 1,
        'dashArray': '1, 1'}).add_to(my_map)
    
    return my_map

my_map_overview=my_map_overview(df_geo)
my_map_overview.save('my_map_overview.html')


In [197]:
# details about the location 

def my_map_details(df_geo):
    my_map=folium.Map(
    location=[48,14],
    tiles='cartodbpositron',
    zoom_start=4,
    width=900,
    height=1000
    )
    
    # instantiate a mark cluster object for the incidents in the dataframe
    incidents = plugins.MarkerCluster().add_to(my_map)
    
    # loop through the dataframe and add each data point to the mark cluster
    for lat, lng, label, in zip(df_geo.latitude, df_geo.logitude, df_geo.Firma):
        folium.CircleMarker(
        location=[lat, lng],
        icon=None,
        popup=label,
        radius=2,color='blue',fill_color='yellow').add_to(incidents)
        
    # add incidents to map
    my_map.add_child(incidents)
    
    # add EU border
    EU_geo = 'I:/EU.geo.json'
    folium.GeoJson(
    EU_geo,
    style_function=lambda feature: {
        'fillColor': '#ffff00',
        'color': 'blue',
        'weight': 1,
        'dashArray': '1, 3'}).add_to(my_map)
    
    # Convert data format
    heatdata = df_geo[['latitude','logitude']].values.tolist()
    
    # add incidents to map
    HeatMap(heatdata).add_to(my_map)
    
    return my_map

my_map_details=my_map_details(df_geo)
my_map_details.save('my_map_details.html')

In [16]:
# get month, dayofweek information 
df_date_geo=df_geo.copy()
df_date_geo['date']= pd.to_datetime(df_date_geo['Datum'], format='%Y-%m-%d')
df_date_geo['month']=df_date_geo['date'].dt.month
df_date_geo['dayofweek']=df_date_geo['date'].dt.dayofweek

df_dayofweek_list = []
for dayofweek in df_date_geo.dayofweek.sort_values().unique():
    df_dayofweek_list.append(df_date_geo.loc[df_date_geo.dayofweek == dayofweek, ['latitude', 'logitude']].values.tolist())

df_month_list = []
for month in df_date_geo.month.sort_values().unique():
    df_month_list.append(df_date_geo.loc[df_date_geo.month == month, ['latitude', 'logitude']].values.tolist())

In [17]:
def my_map_geo_month(df_date_geo):
    my_map=folium.Map(
        location=[48,14],
        tiles='cartodbpositron',   
        zoom_start=5)
    
    latitudes=list(df_date_geo.latitude)
    longitudes=list(df_date_geo.logitude)
    labels=list(df_date_geo.Firma)


    for lat,lng,label in zip(latitudes,longitudes,labels):
        folium.CircleMarker([lat,lng],popup=label,radius=0.5,color='black',fill_color='black').add_to(my_map)
    
    incidents=folium.map.FeatureGroup()
    my_map.add_child(incidents)

    # add EU border


    EU_geo = 'I:/EU.geo.json'
    folium.GeoJson(
    EU_geo,
    style_function=lambda feature: {
        'fillColor': '#ffff00',
        'color': 'gray',
        'weight': 1,
        'dashArray': '1, 3'}).add_to(my_map)

    HeatMapWithTime(df_month_list, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(my_map)
    
    return my_map


In [7]:
def my_map_geo_dayofweek(df_date_geo):
    my_map=folium.Map(
    location=[48,14],
    tiles='cartodbpositron',
    zoom_start=4)

    latitudes=list(df_date_geo.latitude)
    longitudes=list(df_date_geo.logitude)
    labels=list(df_date_geo.Firma)
    
    for lat,lng,label in zip(latitudes,longitudes,labels):
        folium.CircleMarker([lat,lng],popup=label,radius=1,color='black',fill_color='gray').add_to(my_map)

    incidents=folium.map.FeatureGroup()
    my_map.add_child(incidents)

    # add EU border
    EU_geo = 'I:/EU.geo.json'
    folium.GeoJson(
    EU_geo,
    style_function=lambda feature: {
        'fillColor': '#ffff00',
        'color': 'gray',
        'weight': 1,
        'dashArray': '1, 3'}).add_to(my_map)    

    HeatMapWithTime(df_dayofweek_list, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(my_map)
    
    return my_map


In [18]:
df_date=df_date_geo.copy()
df_date['dayofweek']=df_date_geo['dayofweek']+1

#dayofweek_counts.plot(figsize=(12,4))
#month_counts.plot(figsize=(12,4))

In [21]:
# 四个时间段？
def Transfer_timeofday1(timeofday):
    h_timeofday=int(timeofday.hour)
    if (h_timeofday<6) and (h_timeofday>=0):
        timeofday_part='EMorning'
    elif (h_timeofday<12) and (h_timeofday>=6):
        timeofday_part='Morning'
    elif (h_timeofday<18) and (h_timeofday>=12):
        timeofday_part='Afternoon'
    else:
        timeofday_part='Evening'
        
    return timeofday_part


def Time_matters(df_date):
    
    dayofweek_counts=df_date['dayofweek'].value_counts()
    dayofweek_counts=dayofweek_counts.to_frame().sort_index()
    dayofweek_counts['day_index']=['Mo','Tu','We','Th','Fr','Sa','Su']

    month_counts=df_date['month'].value_counts()
    month_counts=month_counts.to_frame().sort_index()
    month_counts['mon_index']=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    
    fig=plt.figure(figsize=[24,9])
    plt.subplot(261)
    sns.pointplot(dayofweek_counts.day_index,dayofweek_counts.dayofweek)


    plt.subplot(263)
    sns.pointplot(month_counts.mon_index,month_counts.month)

    plt.subplot(262) 
    labels=dayofweek_counts.day_index
    X=dayofweek_counts.dayofweek  
    #plt.pie(X,labels=labels,autopct='%1.0f%%') 
    plt.bar(labels,X)
    

    labels=month_counts.mon_index
    X=month_counts.month  
    plt.subplot(264)
    #plt.pie(X,labels=labels,autopct='%1.0f%%') 
    plt.bar(labels,X)

    #--------------------------------------------------------------------

    Hour_counts=df_date['Uhrzeit'].value_counts()
    Hour_counts=Hour_counts.to_frame().sort_index()
    mean_fire=Hour_counts.Uhrzeit.mean()
    Hour_counts['MeanFire']=mean_fire    
    Hour_counts['TwoTimesMeanFire']=mean_fire*2

    ax=fig.add_subplot(2,2,3)
    Hour_counts.plot(color=['B','C1','C2'],grid=True,style=['-','-.','-.'],ax=ax)
    
    #-------------------------------------------------------------------------

    Hour_counts['Uhrzeit_Uhr']=Hour_counts.index
    Hour_counts['TimeofDay']=Hour_counts['Uhrzeit_Uhr'].apply(lambda x: Transfer_timeofday1(x))
    TimeofDay_part=Hour_counts['TimeofDay'].value_counts()
    TimeofDay_part=TimeofDay_part.to_frame().sort_index()

    labels=TimeofDay_part.index
    X=TimeofDay_part.TimeofDay 
    
    plt.subplot(2,6,10)
    #plt.pie(X,labels=labels,autopct='%1.0f%%')
    plt.bar(labels,X)
    
    plotly_plot = plotly.tools.mpl_to_plotly(fig)
    plt.close()
    
    return plotly_plot

Fig_TimeMatters=Time_matters(df_date) 

In [22]:
df_HallMater=data[['Brandentstehungsort','Brandherd','Art des in Brand geratenen Materials','Schadenhöhe']]
df_HallMater.columns=['FireHall','FireSource','FireMaterial','Cost']
df_HallMater.isna().sum()


FireHall         18
FireSource       16
FireMaterial     56
Cost            213
dtype: int64

In [23]:
df_HallMater['FireHall'].value_counts()

Lagerhalle                89
Freilager                 73
Produktion                49
Container                  5
Mehrzweckgebäude           3
unbekannt                  3
Schuppen                   3
Sortierhalle               2
Lagerghalle                2
Bunker                     2
Lagergut                   2
Schredder                  2
Fahrzeug                   2
Lastwagen                  2
Betriebshalle              2
Restmüllcontainer          1
Lagerhalle                 1
Lagerhallen (4)            1
Schredder-Halle            1
Lüftungssystem             1
Überdachter Lagerplatz     1
Lagerhalle (Container)     1
Verladung                  1
Halle                      1
Lahgerhalle                1
Sperrgutbunker             1
Materialbunker             1
Schreddermobil             1
-                          1
Name: FireHall, dtype: int64

In [24]:
# storage matters?
list_rename=['Lagerghalle','Lahgerhalle','Lagerhalle (Container)','Lagerhalle','Lagerhallen (4)']
df_HallMater['FireHall']=df_HallMater['FireHall'].map(lambda x: 'Lagerhalle' if (x in list_rename) else x)

list_rename=['Lagerhalle','Freilager','Produktion']
df_HallMater['FireHall']=df_HallMater['FireHall'].map(lambda x: 'Others' if (x not in list_rename) else x)

In [25]:
df_HallMater['FireSource'].value_counts()
# no information in the 'Brandherd'

Lagergut                           192
Schredder                            9
Förderanlage                         6
Sortieranlage                        6
Presse                               5
unbekannt                            3
Maschine                             3
Kanalballenpresse                    2
Zerkleinerer                         2
-                                    2
Förderband                           2
Flurförderfahrzeug                   1
Luftfilter                           1
Metallgestelle                       1
Hackmaschine                         1
Filteranlage                         1
Papierpresse                         1
Lagergut, Förderanlage               1
Betriebsstoffe                       1
Sortieranlage                        1
Radlader                             1
mobile Sortieranlage                 1
mobiler Schredder                    1
Müllpresswagen                       1
Nebenprodukt                         1
Brenn- und Schmierstoffe 

In [26]:
df_HallMater['FireMaterial'].value_counts()

Papier                                             8
Kunststoff                                         6
Altpapier                                          5
Papierballen                                       5
Kunststoffe                                        5
Elektroschrott                                     5
Restmüll                                           5
Hausmüll                                           4
Sperrmüll                                          4
Metallspäne                                        4
Schrott                                            4
unbekannt                                          2
Ersatzbrennstoffe                                  2
Styroporpaletten                                   2
Metallschrott                                      2
Papierpresse                                       2
Industrieabfall                                    2
Gewerbeabfall                                      2
Sperrmüll, Laub + Gartenabfall                

In [27]:
# map into 11 clusters
Material_clusters   =   ['Batteries','Hausmull','Electronics','Papers','Mix','Plastic','Mineral',
                          'Fluid','Textilien','Metel','others']

Material_clusters_match=['atterien','Straßenkehricht|Müll|Kompost|Hausmüll|Restmüll|Sperrmüll|Biomasse|Kompostabfälle','Elektro|Eiskühlschrank','Papier|papier|Kartonage','Mischabfall|Mischmüll','Plastik|unststoff','Übergriff|Sperrrmüll|Siedlungsabfall|Gewerbemüll|Möbel|Industrieabfall|Altmöbel|Dachplatten|Holz|holz|Bauschutt|Gewerbeabfall|Baumischabfall|Schutt',
                         'ölhaltige|Ersatzbrennstoffe','Teppich|Texti|Filzmatten|Matratzen','Metel|chrott|metall|Magnesium','Schredderanlage|zeuge|wagen|Spraydosen|Holzschredder|Dieseltank|Förder|flug|stapler|maschine|Sortieranlage|Schredder|autos']


In [30]:
Material_cluster_number=[]

for i in list(range(len(Material_clusters))):
    X=np.where(df_HallMater['FireMaterial'].str.contains(Material_clusters_match[i]))
    Material_cluster_number.append(len(X[0]))

Material_cluster=pd.DataFrame(Material_cluster_number,index=Material_clusters,columns =['NumberOfCluster']).sort_values('NumberOfCluster')

In [32]:
fig=plt.figure(figsize=[10,5])

fig.add_subplot(1,3,1)
order=['Lagerhalle','Freilager','Produktion','Others']
df_HallMater['FireHall'].value_counts()[order].plot(kind='bar',title='Ort des Feuers',color='g')
df_HallMater['FireHall'].value_counts()[order].plot(kind='line',title='Ort des Feuers',secondary_y=False,color='r',marker='o')
plt.ylabel('Number of Fires')


fig.add_subplot(1,2,2)
plt.bar(Material_cluster.index, Material_cluster.NumberOfCluster,color='green')
plt.plot(Material_cluster.index, Material_cluster.NumberOfCluster,color='red',marker='o')
plt.ylabel('Number of Fires')
plt.title('Material des Feuers')

Fig_materials = plotly.tools.mpl_to_plotly(fig)
plt.close()

In [229]:
#dash
text_note='''

This is the analysis for the fire of the recycling companies in year 2018-2020.
Total 278 accidents

'''


app=JupyterDash('Results Display',width='1900px',height='1000px')
app.layout=html.Div([
    html.H2('Fire accidents of recycling compines'),
    
    html.Div([
         
        
        html.Div([
            
            dcc.Markdown(children=text_note),
        ],
            style=dict(backgroundColor='gray',height='36%',float='left',width='45%',display='inline-block')
        ),
        
        
        html.Div([
            
            html.Iframe(srcDoc=open('my_map_overview.html','r').read(),height='100%',width='100%'),
        ],
            style=dict(backgroundColor='blue',height='36%',float='right',width='55%',display='inline-block')
        ),
        
        
        html.Iframe(srcDoc=open('my_map_details.html','r').read(),height='70%',width='100%'),
        
    ],
          
        style=dict(height='1000px',width='49%',display='inline-block')
    ),
        
    
    html.Div([
        
        #html.H3('TimeMatters'),
        
        html.Div([
        
        
        
        ],
        style=dict(backgroundColor='yellow',height='500px',width='100%',display='inline-block')
        ),
        
        
        html.H3('MaterialMatters'),
        
        html.Div([
        
        
            
        ],
        style=dict(backgroundColor='red',height='500px',width='100%',display='inline-block')
        ),
        
        
        ],
        
        style=dict(backgroundColor='gray',height='1000px',width='49%',display='inline-block')
    )
    
])

app